My first attmept at implementing linear transformations between coordinate systems - Earth Centered Initial (ECI) to Earth Centred Earth Fixed (ECEF) This is important for navigation in space

In [1]:
import numpy as np

In [ ]:
# create class to group rotation functions

from traitlets import This


class RotationMatrices:
    
    """this classs groups my linear transformation functions"""

    @staticmethod
    def safe_cos(ang_deg):
      ang = np.radians(ang_deg)
      result = np.cos(ang)

      if abs(result) < 1e-15:
        return 0.0
      else:
        return result
       

    @staticmethod
    def rotation_matrix_x(ang_deg):
   
     """This rotates on the x-axis, which means x stays still and y and z rotate around it"""

     ang = np.radians(ang_deg)

     Rx = np.array([
        [1,0,0],
        [0, RotationMatrices.safe_cos(ang_deg), -np.sin(ang)],
        [0, np.sin(ang), RotationMatrices.safe_cos(ang_deg)]
        ])
     
     return Rx

    @staticmethod
    def rotation_matrix_y(ang_deg):
   
     """This rotates on the y-axis, which means y stays still and x and z rotate around it"""

     ang = np.radians(ang_deg)

     Ry = np.array([
        [RotationMatrices.safe_cos(ang_deg),0,np.sin(ang)],
        [0, 1, 0],
        [-np.sin(ang), 0 , RotationMatrices.safe_cos(ang_deg)]
        ])
     
     return Ry
    
    @staticmethod
    def rotation_matrix_z(ang_deg):
   
     """This rotates on the z-axis, which means z stays still and y and x rotate around it"""

     ang = np.radians(ang_deg)

     Rz = np.array([
        [RotationMatrices.safe_cos(ang_deg),-np.sin(ang),0],
        [np.sin(ang), RotationMatrices.safe_cos(ang_deg), 0],
        [0, 0 , 1]
        ])
     
     return Rz
    
    @staticmethod
    def euler_angles_to_rotation_matrix(roll_deg, pitch_deg, yaw_deg, sequence='ZYX'):
      
      """This gives the final rotation matrix given tthe roll, pitch and yaw degree and the desired rotation sequence"""

      roll = np.radians(roll_deg)
      pitch = np.radians(pitch_deg)
      yaw = np.radians(yaw_deg) 

      Rx = RotationMatrices.rotation_matrix_x(roll_deg)
      Ry = RotationMatrices.rotation_matrix_y(pitch_deg)
      Rz = RotationMatrices.rotation_matrix_z(yaw_deg)

      if sequence == 'ZYX':
        return Rx @ Ry @ Rz
      elif sequence == 'XYZ':
        return Rz @ Ry @ Rx
      else:
        raise ValueError(f"Sequence {sequence} not implemented")
      
    @staticmethod
    def rotation_matrix_eci_to_ecef(greenwich_hour_angle_deg):
      """Earth rotates around the z axis so this is simply a z-axis at the greenwich hour angle"""
      
      
      R_eci_to_ecef = RotationMatrices.rotation_matrix_z(greenwich_hour_angle)
      return R_eci_to_ecef
    

      
      

class SpacecraftState:

  """This is my spaceship"""
  def __init__(self, position, velocity, earth_rotation_rate = 15.04):
    """this is where is is and where it is going"""
    self.position = np.array(position)
    self.velocity = np.array(velocity)
    self.earth_rotation_rate = earth_rotation_rate
  

  def rotation(self, rotation_matrix):
    """This is how we manouvre my spaceship in space"""
    self.position = rotation_matrix @ self.position
    self.velocity = rotation_matrix @ self.velocity

  def eci_to_ecef(self, time_hours, eci_position):

    """Apply the rotationn matrix to convert from eci to ecef.
    To do this we need to calculate the greenwich hour angle. We need the time since 
    the reference epoc and the rate of rotation in degrees per hour"""

    gha = self.earth_rotation_rate*time_hours
    ecef_position = np.array(eci_position)

    Reci_to_ecef = RotationMatrices.rotation_matrix_eci_to_ecef(gha)

    ecef_position = Reci_to_ecef @ eci_position

    return ecef_position






In [16]:
# testing out my classes

MySpaceShip = SpacecraftState([1,1,1], [1,1,1], 15.04)

# Let's rotate {degs} degrees around each axis to test the functions, should be unchanged

degs = 45

test_space_ship_position = np.array([1,1,1])

#Testing out my safe_cos fuinction to avoid non-zero values at 90 degrees

print(f"cos = {np.cos(np.radians(degs))}")
print(f"safe_cos = {RotationMatrices.safe_cos(degs)}")

# Calculating rotation matricies for each axis for a {degs} rotation

Rx = RotationMatrices.rotation_matrix_x(degs)
print(f"Rx = {Rx} for a {degs} rotation")

Ry = RotationMatrices.rotation_matrix_y(degs)
print(f"Ry = {Ry} for a {degs} rotation")

Rz = RotationMatrices.rotation_matrix_z(degs)
print(f"Rz = {Rz} for a {degs} rotation")

# Applying rotation matricies to my spaceship position on axis at a time
print(f" myspaceship position after a {degs}  rotation on the X-axis is  {Rx @ test_space_ship_position}")

print(f" myspaceship position after a {degs}  rotation on the Y-axis is  {Ry @ test_space_ship_position}")

print(f" myspaceship position after a {degs}  rotation on the Z-axis is  {Rz @ test_space_ship_position}")

# Let's compare the appraoch of rotating each axis separtely and composing the matrix of column i, j and k and using matrix multiplication in 
# Yaw, pitch, roll sequence, so if I multiply each rotation matrix bu its corresponding basis vector I should get the same result as matrix multiplication
yaw_deg = 0
pitch_deg = 90
roll_deg = 0

print(f" myspaceship position after a {yaw_deg}  yaw, {pitch_deg} pitch and {roll_deg} roll rotation is  {RotationMatrices.euler_angles_to_rotation_matrix(roll_deg, pitch_deg, yaw_deg) @ test_space_ship_position}")


# And now we rotate from ECI to ECEF
ECEF_position = SpacecraftState.eci_to_ecef(SpacecraftState, 5, [1,1,1])

print (f"ECEF position is {ECEF_position}")


cos = 0.7071067811865476
safe_cos = 0.7071067811865476
Rx = [[ 1.          0.          0.        ]
 [ 0.          0.70710678 -0.70710678]
 [ 0.          0.70710678  0.70710678]] for a 45 rotation
Ry = [[ 0.70710678  0.          0.70710678]
 [ 0.          1.          0.        ]
 [-0.70710678  0.          0.70710678]] for a 45 rotation
Rz = [[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]] for a 45 rotation
 myspaceship position after a 45  rotation on the X-axis is  [1.00000000e+00 1.11022302e-16 1.41421356e+00]
 myspaceship position after a 45  rotation on the Y-axis is  [1.41421356e+00 1.00000000e+00 1.11022302e-16]
 myspaceship position after a 45  rotation on the Z-axis is  [1.11022302e-16 1.41421356e+00 1.00000000e+00]
 myspaceship position after a 0  yaw, 90 pitch and 0 roll rotation is  [ 1.  1. -1.]


AttributeError: type object 'SpacecraftState' has no attribute 'earth_rotation_rate'